In [1]:
import pandas as pd
import altair as alt

In [2]:
results_df = pd.read_csv("../data/interim/results_table.csv")

In [3]:
results_df["Score: "] = pd.to_numeric(results_df["Score: "]) 
results_df = results_df.rename(columns={"Score: ": 'Score', 'Test Set:': 'Test Set', 'Race:':'Race', 
                                        "Model: ":"Model", "Metric:": "Metric"})
results_df["Metric"] = results_df["Metric"].apply(lambda x: "F1 Weighted Average" if str(x) == "F1-weighted-avg"else x)
results_df["Race"] = results_df["Race"].apply(lambda x: str(x).title() if str(x) != 'api' else str(x).upper())

luke_baseline_results = results_df.loc[(results_df["Model"] == "LUKE-Baseline") & (results_df["Test Set"] != "CoNLL-03 Test")]
luke_custom_results = results_df.loc[(results_df["Model"] == "LUKE-Custom") & (results_df["Test Set"] != "CoNLL-03 Test")]
flair_baseline_results = results_df.loc[(results_df["Model"] == "Flair-Baseline") & (results_df["Test Set"] != "CoNLL-03 Test")]

In [4]:
def plot_breakdown(input_df, title):
    groups = ["Best Performing Race Group", "Other 3 Race Groups"]
    scores = [list(input_df.loc[input_df["Race"]=="White"]["Score"])[0], input_df.loc[input_df["Race"]!="White"]["Score"].mean()]
    
    df = pd.DataFrame(columns=["Group", "F1 Score"])
    df["Mean F1 Score"] = scores
    df["Group"] = groups
    
    chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Group:O', axis=alt.Axis(labelAngle=0, ticks=False), sort={"encoding":"y", "order":"descending"}),
    y=alt.Y('Mean F1 Score:Q', axis=alt.Axis(labelAngle=0))
    ).properties(width = 300, height = 300, title=title)
    
    text = chart.mark_text(
        align='center',
        baseline='top',
        dy=5, # update
        color='white'
    ).encode(
        text='Mean F1 Score:Q',
    )

    return_chart = (chart + text).configure_axis(grid=False).properties(title="CoNLL-2003 Test Results").properties(width = 300, height = 300, title=title)
    return return_chart

#     return chart

In [5]:
plot_breakdown(luke_baseline_results, "Baseline LUKE Results")

alt.LayerChart(...)

In [6]:
plot_breakdown(luke_custom_results, "Custom LUKE Results")

alt.LayerChart(...)

In [7]:
plot_breakdown(flair_baseline_results, "Baseline Flair Results")

alt.LayerChart(...)

In [8]:
def plot_all_group_chart(input_df, title):
    sort_order = ["White", "Hispanic", "API", "Black"]

    chart = alt.Chart(input_df).mark_bar().encode(
        x=alt.X('Race:O', axis=alt.Axis(labelAngle=0, ticks=False), sort=sort_order),
        y=alt.Y('Score:Q', axis=alt.Axis(labelAngle=0), title="F1 Score")
    )

    text = chart.mark_text(
        align='center',
        baseline='top',
        dy=5, # update
        color='white'
    ).encode(
        text='Score:Q',
    )

    return_chart = (chart + text).configure_axis(grid=False).properties(title="CoNLL-2003 Test Results").properties(width = 300, height = 300, title=title)
    return return_chart

In [9]:
luke_baseline_chart = plot_all_group_chart(luke_baseline_results, "Baseline LUKE Results")
luke_custom_chart = plot_all_group_chart(luke_custom_results, "Custom LUKE Results")
flair_baseline_chart = plot_all_group_chart(flair_baseline_results, "Baseline Flair Results")

In [10]:
luke_baseline_chart

alt.LayerChart(...)

In [11]:
luke_custom_chart

alt.LayerChart(...)

In [12]:
flair_baseline_chart

alt.LayerChart(...)

In [13]:
def plot_conll_results(input_df):
    conll_results = input_df.loc[(input_df["Test Set"] == "CoNLL-03 Test")]

    sort_order = ["Flair-Baseline", "LUKE-Baseline", "LUKE-Custom"]
    bars = alt.Chart(data=conll_results).mark_bar().encode(
        x=alt.X('Model:N', axis=alt.Axis(labelAngle=0, ticks=False), sort=sort_order),
        y=alt.Y('Score:Q', axis=alt.Axis(labelAngle=0), title="F1 Score"), 
        color=alt.Color("Model:N")).properties(
        height=300,
        width=225
    )

    text = bars.mark_text(
        align='center',
        baseline='top',
        dy=-10, # update
        color='white').encode(text='Score:Q')

    chart = alt.layer(bars, text, data=conll_results).facet(
        column=alt.Column("Metric:N", title="Metric")).configure_axis(grid=False).properties(title="CoNLL-2003 Test Results")
    
    return chart


In [14]:
conll_results = plot_conll_results(results_df)

In [15]:
conll_results

alt.FacetChart(...)